In [2]:
import os, glob
import cv2
import numpy as np
import sympy
from sympy import symbols

In [3]:
Root = os.getcwd()
os.makedirs('Images_Folder', exist_ok=True)
os.makedirs('Result_Folder', exist_ok=True)

In [4]:
def Convert_to_PNG():
    os.chdir(os.path.join(Root,'Images_Folder'))
    image_patterns = ["*.jpeg", "*.jpg"]
    for pattern in image_patterns:
        for image_file in glob.glob(pattern):
            if not image_file.endswith(".png"):
                new_file = os.path.splitext(image_file)[0] + ".png"
                os.rename(image_file, new_file)
    os.chdir(Root)

In [5]:
def Convert_Image_To_Matrix(Image_Name):
    if Image_Name in os.listdir('Images_Folder'):
        Image = cv2.imread(os.path.join('Images_Folder', Image_Name))
        Gray_Image = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
        Data = np.array(Gray_Image)/255 # Chuẩn hóa dữ liệu
        return Data
    else: print("Picture not found!!!!")

In [6]:
def Create_Zero_Matrix(Values):
    Zero_Matrix = np.zeros((Values.shape[0], Values.shape[1]))
    return Zero_Matrix

def Create_Identity_Matrix(Matrix):
    Identity_Matrix = Create_Zero_Matrix(Matrix)
    for Row_Column in range(Identity_Matrix.shape[1]):
        Identity_Matrix[Row_Column,Row_Column] = 1
    return Identity_Matrix

In [ ]:
class Calculation:
    def __init__(self, Name, Matrix_Approximation):
        self.Matrix = Convert_Image_To_Matrix(Name)
        self.k = Matrix_Approximation
        
if __name__ == '__main__':
    Calculation('Cute Cat.png', 5)